## Load packages
- numpy, tensorflow, matplotlib

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline

slim = tf.contrib.slim

## Load Meow Dataset
- 64x64 meow data

In [2]:
meow_data_path = './data/meow_64.npz'
meow_dataset = np.load(meow_data_path)

print(meow_dataset.files)

# parse dataset
X_train = meow_dataset['trainimg']
X_test = meow_dataset['testimg']
X_ = np.append(X_train, X_test, axis=0)

print("Training images: ", X_train.shape)
print("Test images: ", X_test.shape)
print("All images: ", X_.shape)

['trainimg', 'trainlabel', 'testimg', 'testlabel', 'imgsize', 'use_gray', 'categories']
Training images:  (3647, 12288)
Test images:  (912, 12288)
All images:  (4559, 12288)


In [3]:
n_data = X_.shape[0]
img_dim = meow_dataset['imgsize']

X = np.reshape(X_, (n_data, img_dim[0], img_dim[1], 3))
print("data shape: ", X.shape)

data shape:  (4559, 64, 64, 3)


# DCGAN
## Generator

In [4]:
colors = 3
g_sizes = {
    'z': 100,
    'conv_layers': [
        (512, 5, 2, True),
        (256, 5, 2, True),
        (128, 5, 2, True),
        (64, 5, 2, True),
        (colors, 5, 2, False)
    ],
    'dense_layers': [],
    'output_activation': tf.tanh,
}

In [5]:
conv_layers = g_sizes['conv_layers']

len(conv_layers)

5

In [6]:
def generator(Z, img_length, g_sizes, is_training=True):
    # dimensions
    latent_dim = g_sizes['z']

    with tf.variable_scope("generator"):
        # determine the size of the data at each step
        dims = [img_length]
        dim = img_length
        n_conv_layers = len(g_sizes['conv_layers'])

        for _, _, stride, _ in reversed(g_sizes['conv_layers'][1:]):
            dim = int(np.ceil(float(dim) / stride))
            dims.append(dim)

        dims = list(reversed(dims))
        print("dims: ", dims)
        g_dims = dims

        # output may use tanh or sigmoid
        num_relues = n_conv_layers - 1
        activation_functions = [tf.nn.relu]*num_relues + [g_sizes['output_activation']]

        print('---- generator net ----')
        for i in range(n_conv_layers):
            # generator network setting 
            name = "deconv_layer_%s" % i
            mo, filter_size, stride, apply_batch_norm = g_sizes['conv_layers'][i]
            f = activation_functions[i]
            output_shape = [None, dims[i], dims[i], mo]
            print("output shape:", output_shape)

            # build generator network
            if i == 0:
                net = slim.fully_connected(Z, mo*dims[i]*dims[i])
                net = tf.reshape(net, (-1, dims[i], dims[i], mo))
            else:
                net = slim.conv2d_transpose(inputs=net, num_outputs=mo,
                                            kernel_size=filter_size, stride=stride, 
                                            activation_fn=f, scope=name)
            if apply_batch_norm:
                net = slim.batch_norm(net, is_training=is_training, scope=name+'_bn')
            mi = mo

        return net

In [7]:
tf.reset_default_graph()

img_length = 64
Z = tf.placeholder(tf.float32, shape=(None, 100), name='Z')

sample_images = generator(Z, img_length, g_sizes)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

sample_z = np.random.random((10,100))
print(sample_z.shape)

test_X_hat = sess.run(sample_images, feed_dict={Z: sample_z})
print(test_X_hat.shape)

dims:  [4, 8, 16, 32, 64]
---- generator net ----
output shape: [None, 4, 4, 512]
output shape: [None, 8, 8, 256]
output shape: [None, 16, 16, 128]
output shape: [None, 32, 32, 64]
output shape: [None, 64, 64, 3]
(10, 100)
(10, 64, 64, 3)


## Discriminator

In [8]:
d_sizes = {
    'conv_layers': [
        (64, 5, 2, False),
        (128, 5, 2, True),
        (256, 5, 2, True),
        (512, 5, 2, True)
    ],
    'dense_layers': [],
  }

In [9]:
x = tf.placeholder(tf.float32, (None, img_length, img_length, 3), name='x')
is_training = True

# def discriminator(x, img_length, d_sizes):
'''build discriminator'''
mi = 3
dim = img_length

for i, (mo, filter_size, stride, apply_batch_norm) in enumerate(d_sizes['conv_layers']):
    name = 'conv_%s' % i
#     print(mo, filter_size, stride, apply_batch_norm)
    with slim.arg_scope([slim.conv2d], padding='SAME',
                       weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                       weights_regularizer=slim.l2_regularizer(0.0005),
                       activation_fn=tf.nn.relu):
        if i == 0:
            net = slim.conv2d(x, mo, [filter_size, filter_size], stride=stride, scope=name)
        else:
            net = slim.conv2d(net, mo, [filter_size, filter_size], stride=stride, scope=name)
        if apply_batch_norm:
            net = slim.batch_norm(net, is_training=is_training, scope=name+'_bn')
        print(name, "shape : ", net.shape)
        
net = slim.flatten(net)
logits = slim.fully_connected(net, 1, scope="fc", activation_fn=None)
print("fc shape: ", logits.shape)
        



64 5 2 False
conv_0 shape :  (?, 32, 32, 64)
128 5 2 True
conv_1 shape :  (?, 16, 16, 128)
256 5 2 True
conv_2 shape :  (?, 8, 8, 256)
512 5 2 True
conv_3 shape :  (?, 4, 4, 512)
fc shape:  (?, 1)


In [10]:
net

<tf.Tensor 'Flatten/Reshape:0' shape=(?, 8192) dtype=float32>

In [11]:
logits

<tf.Tensor 'fc/BiasAdd:0' shape=(?, 1) dtype=float32>

In [12]:
test_input = np.random.random((10,64,64,3))
print(test_input.shape)

(10, 64, 64, 3)


In [13]:
test_inputs = np.random.random((10,64,64,3))
sess.run(tf.global_variables_initializer())
logits_ = sess.run(logits, feed_dict={x: test_inputs})

In [15]:
print(logits_)

[[-1.01259434]
 [ 0.50866759]
 [ 0.03478894]
 [ 0.75325912]
 [-0.23173884]
 [ 1.11670589]
 [-0.86125094]
 [-0.16755843]
 [ 0.80265462]
 [ 2.14541936]]
